In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import lightgbm as lgb
from tqdm.notebook import tqdm
from sklearn.model_selection import ParameterGrid
from xgboost import XGBRegressor

In [ ]:
df = pd.read_csv('D:\\NCKH - Estimation GHG\\Output\\2020\\data_co2_modis.csv')
df.info()

In [ ]:
df.describe()

In [ ]:
df_drop = df.drop(columns=['LST_Night_Terra_C', 'LST_Night_Aqua_C'])
df_dropna = df_drop.dropna()
df_dropna.describe()

In [ ]:
feature_columns = df_dropna.columns[df_dropna.columns.get_loc('precipitation'):].tolist()
target = ['xco2']

In [ ]:
df_selected = df_dropna[feature_columns + target]
correlation_with_target = df_selected.corr()['xco2'].drop('xco2')
print(correlation_with_target)

In [ ]:
from sklearn.model_selection import train_test_split

X = df_selected[feature_columns]
y = df_selected['xco2']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Kích thước tập huấn luyện: {X_train.shape}, {y_train.shape}")
print(f"Kích thước tập kiểm tra: {X_test.shape}, {y_test.shape}")

In [ ]:
param_grid_rf = {
    'n_estimators': [250, 300, 350, 400],      
    'max_features': ['sqrt', 'log2', 0.7], 
    'max_depth': [None, 10, 20, 30],       
    'min_samples_split': [2, 5, 10],      
    'min_samples_leaf': [1, 2, 4],         
    'bootstrap': [True]                    
}

rf = RandomForestRegressor(random_state=42, n_jobs=-1) 

grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf,
                              cv=5, 
                              # scoring='neg_mean_squared_error', 
                              scoring='r2',
                              verbose=2, 
                              n_jobs=-1) 

grid_search_rf.fit(X_train, y_train)

print("\nCác siêu tham số tốt nhất cho Random Forest:")
print(grid_search_rf.best_params_)

best_rf = grid_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print(f"\nĐánh giá Random Forest trên tập kiểm tra:")
print(f"  Mean Squared Error (MSE): {mse_rf:.4f}")
print(f"  R-squared (R2): {r2_rf:.4f}")

In [ ]:
param_grid_xgb = {
    'n_estimators': [250, 300, 350, 400],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.7, 0.8, 0.9, 1.0],          
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],   
    'gamma': [0, 0.1, 0.2],                     
    'reg_alpha': [0, 0.001, 0.01],            # L1 regularization 
    # 'reg_lambda': [1, 0.1, 0.01],           # L2 regularization 
    'tree_method': ['hist'],
    'device': ['cuda']
}

grid = list(ParameterGrid(param_grid_xgb))
best_score = -float('inf')
best_model = None
best_params = None

for params in tqdm(grid, desc="Tuning models"):
    model = XGBRegressor(
        objective='reg:squarederror',
        random_state=42,
        n_jobs=-1,
        **params
    )
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test) 
    if score > best_score:
        best_score = score
        best_model = model
        best_params = params

print("\nCác siêu tham số tốt nhất cho XGBoost:")
print(best_params)

y_pred = best_model.predict(X_test)
mse_xg = mean_squared_error(y_test, y_pred)
r2_xg = r2_score(y_test, y_pred)

print("\nĐánh giá mô hình XGBoost trên tập kiểm tra:")
print(f"  Mean Squared Error (MSE): {mse_xg:.4f}")
print(f"  R-squared (R2): {r2_xg:.4f}")